<a href="https://colab.research.google.com/github/JHyunjun/Attention/blob/main/TF2_0_AttentionGRU_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.03.08

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
#xy_test = xy_train

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  print(i)
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)

In [ ]:
# Data Sequence Setting
TIME_STEPS = 5
seq_length = TIME_STEPS

train_set = xy_train_scale
test_set = xy_test_scale

def build_dataset(time_series, seq_length) : 
  dataX = []
  dataY = []
  for i in range(0, len(time_series) -seq_length) : 
    x = time_series[i:i+seq_length , : :]
    y = time_series[i + seq_length -1, :]
    dataX.append(x)
    dataY.append(y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print(trainX.shape, trainY.shape) #(# of datas, Time_steps, features)

INPUT_DIM = trainX.shape[2]

In [ ]:
#AttentionGRUAE NEW

def attention_layer_encoder(inputs) :
  input_dim = int(inputs.shape[2])
  a = tf.keras.layers.Permute((2,1))(inputs)
  a = tf.keras.layers.Dense(TIME_STEPS, activation = 'softmax')(a)
  a_probs = tf.keras.layers.Permute((2,1), name = 'attention_vec_encoder')(a)
  output_attention_mul = tf.keras.layers.multiply([inputs, a_probs])

  return output_attention_mul

def attention_layer_decoder(inputs) :
  input_dim = int(inputs.shape[2])
  a = tf.keras.layers.Permute((2,1))(inputs)
  a = tf.keras.layers.Dense(TIME_STEPS, activation = 'softmax')(a)
  a_probs = tf.keras.layers.Permute((2,1), name = 'attention_vec_decoder')(a)
  output_attention_mul = tf.keras.layers.multiply([inputs, a_probs])

  return output_attention_mul

def Attention_GRU_AE() : 
  # Encoding
  inputs = tf.keras.Input(shape = (TIME_STEPS, INPUT_DIM, ))
  gru_units = 32
  gru_out = tf.keras.layers.GRU(gru_units, return_sequences = True)(inputs)
  gru_out = tf.keras.layers.GRU(gru_units, return_sequences = True)(gru_out)
  encoder_output = attention_layer_encoder(gru_out)
  # Decoding
  decoder_gru_units = 32
  decoder_gru_out = tf.keras.layers.GRU(decoder_gru_units, return_sequences = True)(encoder_output)
  decoder_gru_out = tf.keras.layers.GRU(decoder_gru_units, return_sequences = True)(decoder_gru_out)
  attention_decoder = attention_layer_decoder(decoder_gru_out)
  attention_decoder = tf.keras.layers.Flatten()(attention_decoder)
  output = tf.keras.layers.Dense(INPUT_DIM, activation = 'sigmoid')(attention_decoder)
  model = tf.keras.Model(inputs = [inputs], outputs = output)

  return model

In [ ]:
#RUN
modelhj = Attention_GRU_AE()
modelhj.compile(optimizer = 'adam', loss = 'mse')
modelhj.fit(trainX, trainY, epochs = 3, batch_size = 1, validation_split = 0)

In [ ]:
# Training
trainY_predict = modelhj.predict(trainX)
print(trainY_predict.shape)

In [ ]:
trainY_predict_back = HJ_backMinMax(trainY_predict)
trainY_back = HJ_backMinMax(trainY)

plt.figure(figsize = (12,16))
plt.subplot(2,2,1)
plt.plot(trainY_back[seq_length:,0], color = 'blue')
plt.plot(trainY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(trainY_back[seq_length:,1], color = 'blue')
plt.plot(trainY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(trainY_back[seq_length:,2], color = 'blue')
plt.plot(trainY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.plot(trainY_back[seq_length:,3], color = 'blue')
plt.plot(trainY_predict_back[:,3], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,4)
plt.plot(trainY_back[seq_length:,4], color = 'blue')
plt.plot(trainY_predict_back[:,4], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])

In [ ]:
# Training
testY_predict = modelhj.predict(testX)
print(testY_predict.shape)

In [ ]:
testY_predict_back = HJ_backMinMax(testY_predict)
testY_back = HJ_backMinMax(testY)

plt.figure(figsize = (12,16))
plt.subplot(2,2,1)
plt.plot(testY_back[seq_length:,0], color = 'blue')
plt.plot(testY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(testY_back[seq_length:,1], color = 'blue')
plt.plot(testY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(testY_back[seq_length:,2], color = 'blue')
plt.plot(testY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.plot(testY_back[seq_length:,3], color = 'blue')
plt.plot(testY_predict_back[:,3], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,4)
plt.plot(testY_back[seq_length:,4], color = 'blue')
plt.plot(testY_predict_back[:,4], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])